In [6]:
import os

import numpy as np


from pydrake.geometry import StartMeshcat
from pydrake.visualization import ModelVisualizer
from pydrake.systems.framework import DiagramBuilder
from pydrake.multibody import inverse_kinematics
from manipulation.station import MakeHardwareStation, load_scenario
from manipulation.scenarios import AddIiwaDifferentialIK, ExtractBodyPose, MakeManipulationStation
from manipulation.meshcat_utils import AddMeshcatTriad, PublishPositionTrajectory
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    Simulator,
    CompositeTrajectory,
    RigidTransform,
    ModelInstanceIndex,
    MultibodyPlant,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    Parser,
    RollPitchYaw,
    RotationMatrix,
    Solve,
    PiecewisePolynomial,
    Trajectory,
    TrajectorySource,
    KinematicTrajectoryOptimization,
    BsplineTrajectory,
    PositionConstraint,
    OrientationConstraint,
    Role,
    PathParameterizedTrajectory,
    MinimumDistanceLowerBoundConstraint
)
from pydrake.visualization import MeshcatPoseSliders
from manipulation.meshcat_utils import WsgButton

In [7]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7002


In [30]:
tree_model_name = "tree"
your_model_filename = "/Users/udayan/Documents/SEAS/6.4212/6.4212Project/TreeGeneration/tree.sdf" # Write the absolute path to your file here
if your_model_filename:
    visualizer = ModelVisualizer(meshcat=meshcat)
    visualizer.parser().AddModels(your_model_filename)
    visualizer.Run(loop_once=True)

In [75]:
scenario_data = f"""
directives:
- add_model:
    name: {tree_model_name}
    file: file://{your_model_filename}
    default_free_body_pose:
        branch_a: # Change here!
            translation: [0, 0, 1]
            rotation: !Rpy {{ deg: [0, 0, 0] }}
- add_frame:
    name: tree_on_world
    X_PF:
        base_frame: world
        translation: [0, 0, 0.5]
        rotation: !Rpy {{ deg: [0, 0, 0] }}
- add_weld:
    parent: tree_on_world
    child: {tree_model_name}::branch_a
- add_model:
      name: iiwa
      file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
      default_joint_positions:
        iiwa_joint_1: [-0.2]
        iiwa_joint_2: [0.79]
        iiwa_joint_3: [0.32]
        iiwa_joint_4: [-1.76]
        iiwa_joint_5: [-0.36]
        iiwa_joint_6: [0.64]
        iiwa_joint_7: [-0.73]
- add_frame:
    name: iiwa_on_world
    X_PF:
      base_frame: world
      translation: [0.3, -0.9, 0.4]
      rotation: !Rpy {{ deg: [0, 0, 90] }}
- add_weld:
    parent: iiwa_on_world
    child: iiwa::iiwa_link_0
- add_model:
    name: wsg
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
    default_joint_positions:
      left_finger_sliding_joint: [-0.02]
      right_finger_sliding_joint: [0.02]
- add_frame:
    name: wsg_on_iiwa
    X_PF:
      base_frame: iiwa_link_7
      translation: [0, 0, 0.114]
      rotation: !Rpy {{ deg: [90, 0, 90] }}
- add_weld:
    parent: wsg_on_iiwa
    child: wsg::body
"""

In [76]:
meshcat.Delete()
builder = DiagramBuilder()

# New Manipulation Station Approach
station = builder.AddSystem(MakeManipulationStation(model_directives=scenario_data, time_step=0.001))
visualizer = MeshcatVisualizer.AddToBuilder(
    builder,
    station.GetOutputPort("query_object"),
    meshcat,
    MeshcatVisualizerParams(delete_on_initialization_event=False),
)

initial_run = 3

# From default joint positions above. Should remove redunduancy by just calling SetDefaultPositions on the plant
q0 = np.array([-0.2, 0.79, 0.32, -1.76, -0.36, 0.64, -0.73])
initial_trajectory = PiecewisePolynomial.FirstOrderHold([0., initial_run], np.array([q0, q0]).T)
q_traj_system = builder.AddSystem(TrajectorySource(initial_trajectory))
builder.Connect(
    q_traj_system.get_output_port(), station.GetInputPort("iiwa_position")
)
initial_g_traj = PiecewisePolynomial.ZeroOrderHold([0., initial_run], np.array([[0.1], [0.1]]).T)
g_traj_system = builder.AddSystem(TrajectorySource(initial_g_traj))
builder.Connect(
    g_traj_system.get_output_port(), station.GetInputPort("wsg_position")
)

plant = station.GetSubsystemByName("plant")

diagram = builder.Build()

diagram_context = diagram.CreateDefaultContext()
station_context = station.GetMyContextFromRoot(diagram_context)
plant_context = plant.GetMyContextFromRoot(diagram_context)

simulator = Simulator(diagram, diagram_context)
#simulator.AdvanceTo(10)

port = station.GetInputPort("iiwa_position")
port.Eval(station_context)

q_traj_system_context = q_traj_system.GetMyContextFromRoot(diagram_context)

meshcat.StartRecording()
simulator.AdvanceTo(initial_run)
#meshcat.PublishRecording()

port = q_traj_system.GetOutputPort("y0")
print(port.Eval(q_traj_system_context))

iiwa_model = plant.GetModelInstanceByName("iiwa")
q0 = plant.GetPositions(plant_context, iiwa_model)
print(q0)

[-0.2   0.79  0.32 -1.76 -0.36  0.64 -0.73]
[-0.19999834  0.79000166  0.32000471 -1.75999481 -0.36008229  0.64006419
 -0.72993016]


In [77]:
def get_apple_body_names():
    tree_index = plant.GetModelInstanceByName(tree_model_name)
    apple_body_names = []
    for i in plant.GetBodyIndices(tree_index):
        body_name = plant.get_body(i).name()
        if body_name.startswith('apple_'):
            apple_body_names.append(body_name)
    return apple_body_names

name = ""

def find_closest_apples_to_iiwa():
    iiwa_body = plant.GetBodyByName("left_finger")
    X_WI = plant.EvalBodyPoseInWorld(plant_context, iiwa_body)

    apple_body_names = get_apple_body_names()
    min_distance = np.inf
    apples = []
    for apple_body_name in apple_body_names:
        apple_body = plant.GetBodyByName(apple_body_name)
        X_WA = plant.EvalBodyPoseInWorld(plant_context, apple_body)
        W_IA = X_WI.inverse() @ X_WA
        distance = np.linalg.norm(W_IA.translation())
        apples.append((apple_body_name, W_IA, distance))
        '''
        if distance < min_distance:
            min_distance = distance
            best_apple = apple_body
            name = apple_body_name
    return best_apple
        '''
    apples = sorted(apples, key=lambda apple: apple[2])
    return apples

apples = find_closest_apples_to_iiwa()
best_apple_name = apples[1][0]
best_apple_body = plant.GetBodyByName(best_apple_name)

X_WA = plant.EvalBodyPoseInWorld(plant_context, best_apple_body)

AddMeshcatTriad(meshcat, "apple", length=0.25, radius=0.01, X_PT=X_WA)
iiwa_body = plant.GetBodyByName("left_finger")
X_WI = plant.EvalBodyPoseInWorld(plant_context, iiwa_body)

frame = plant.GetFrameByName("wsg_on_iiwa")
X_WI = frame.CalcPoseInWorld(plant_context)

AddMeshcatTriad(meshcat, "iiwa", length=0.25, radius=0.01, X_PT=X_WI)

In [78]:
def CreateIiwaControllerPlant(visualize=False):
    """creates plant that includes only the robot and gripper, used for controllers."""
    robot_sdf_path = "package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf"
    #gripper_sdf_path = "package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf"
    gripper_sdf_path = "package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_welded_fingers.sdf"
    sim_timestep = 1e-3
    #plant_robot = MultibodyPlant(sim_timestep)
    builder = DiagramBuilder()
    plant_robot, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=sim_timestep)
    parser = Parser(plant=plant_robot)
    parser.AddModelsFromUrl(robot_sdf_path)
    parser.AddModelsFromUrl(gripper_sdf_path)
    # NOTE FOR ME-  CAN USE ADD MODELS FROM STRING
    plant_robot.WeldFrames(
        frame_on_parent_F=plant_robot.world_frame(),
        frame_on_child_M=plant_robot.GetFrameByName("iiwa_link_0"),
        X_FM=RigidTransform(
            RollPitchYaw(0, 0, np.pi / 2), np.array([0.3, -0.9, 0.4])
        ),
    )
    plant_robot.WeldFrames(
        frame_on_parent_F=plant_robot.GetFrameByName("iiwa_link_7"),
        frame_on_child_M=plant_robot.GetFrameByName("body"),
        X_FM=RigidTransform(
            RollPitchYaw(np.pi / 2, 0, np.pi / 2), np.array([0, 0, 0.114])
        ),
    )
    plant_robot.mutable_gravity_field().set_gravity_vector([0, 0, 0])

    if visualize:
        #meshcat.Delete()
        visualizer = MeshcatVisualizer.AddToBuilder(
            builder,
            scene_graph,
            meshcat,
            MeshcatVisualizerParams(role=Role.kIllustration),
        )
    else:
        visualizer = None

    plant_robot.Finalize()
    diagram = builder.Build()

    plant_robot.SetDefaultPositions(q0)

    return plant_robot, visualizer, diagram


In [79]:
def find_trajectory(q_0, X_WTarget, control_points = 10, q_guess_final = None):

    visualize = False
    
    q_knots = []
    local_plant, visualizer, diagram = CreateIiwaControllerPlant(visualize)

    context = diagram.CreateDefaultContext()
    local_plant_context = local_plant.GetMyContextFromRoot(context)
    
    world_frame = local_plant.world_frame()

    wsg = local_plant.GetModelInstanceByName("Schunk_Gripper")

    num_q = local_plant.num_positions()
    q0 = local_plant.GetPositions(local_plant_context)
    gripper_frame = local_plant.GetFrameByName("body", wsg)

    trajopt = KinematicTrajectoryOptimization(local_plant.num_positions(), control_points)
    prog = trajopt.get_mutable_prog()

    # Todo should I attempt better or different guesses
    if q_guess_final is not None:
        q_guess = np.linspace(q_0.squeeze(), q_guess_final, trajopt.num_control_points()).T
    else:
        q_guess = np.tile(q_0.reshape((7, 1)), (1, trajopt.num_control_points()))
    #q_guess[0, :] = np.linspace(0, -np.pi / 2, trajopt.num_control_points())
    print(trajopt.num_control_points())
    print(q_guess.shape)
    
    path_guess = BsplineTrajectory(trajopt.basis(), q_guess)
    trajopt.SetInitialGuess(path_guess)

    # Uncomment this to see the initial guess:
    #PublishPositionTrajectory(path_guess, context, plant, visualizer)

    
    trajopt.AddDurationCost(1.0)
    trajopt.AddPathLengthCost(1.0)
    trajopt.AddPositionBounds(
        local_plant.GetPositionLowerLimits(), local_plant.GetPositionUpperLimits()
    )
    trajopt.AddVelocityBounds(
        local_plant.GetVelocityLowerLimits()/5, local_plant.GetVelocityUpperLimits()/5
    )
    
    trajopt.AddDurationConstraint(1, 10)

    # start constraint
    trajopt.AddPathPositionConstraint(lb=q_0, ub=q_0, s=0)

    # goal constraint
    goal_position_constraint = PositionConstraint(
        local_plant,
        local_plant.world_frame(),
        X_WTarget.translation(),
        X_WTarget.translation(),
        gripper_frame,
        [0, 0.1, 0],
        local_plant_context,
    )
    trajopt.AddPathPositionConstraint(goal_position_constraint, 1)
    goal_orientation_constraint = OrientationConstraint(
        local_plant,
        local_plant.world_frame(),
        X_WTarget.rotation(),
        gripper_frame,
        RotationMatrix(),
        np.pi/180 * 1,
        local_plant_context,
    )
    trajopt.AddPathPositionConstraint(goal_orientation_constraint, 1)

    prog.AddQuadraticErrorCost(
        np.eye(num_q), q0, trajopt.control_points()[:, -1]
    )

    # start and end with zero velocity
    trajopt.AddPathVelocityConstraint(
        np.zeros((num_q, 1)), np.zeros((num_q, 1)), 0
    )
    trajopt.AddPathVelocityConstraint(
        np.zeros((num_q, 1)), np.zeros((num_q, 1)), 1
    )

    # collision constraints
    '''
    collision_constraint = MinimumDistanceLowerBoundConstraint(
        plant, 0.001, plant_context, None, 0.01
    )
    evaluate_at_s = np.linspace(0, 1, 50)
    for s in evaluate_at_s:
        trajopt.AddPathPositionConstraint(collision_constraint, s)
    '''

    result = Solve(prog)
    if not result.is_success():
        print("Trajectory optimization failed")
        print(result.get_solver_id().name())

    trajectory = trajopt.ReconstructTrajectory(result)
    
    if (visualize):
        PublishPositionTrajectory(
            trajectory, context, local_plant, visualizer
        )
    
    return trajectory

In [81]:
#best_apple_name = apples[7][0]
best_apple_name = apples[3][0]
best_apple_body = plant.GetBodyByName(best_apple_name)

X_WA = plant.EvalBodyPoseInWorld(plant_context, best_apple_body)

def convertApplePoseToGrasp(X_WA):
    #X_AG = RigidTransform(RotationMatrix.MakeXRotation(-np.pi/2), [0, 0, 0.03])
    X_AG = RigidTransform(RotationMatrix.MakeXRotation(-np.pi/2), [0, 0, 0.03])
    X_WG = X_WA @ X_AG
    #rotation = RigidTransform(RotationMatrix.MakeYRotation(np.pi/4 * 7))
    #X_WG = X_WG @ rotation
    return X_WG

def convertGraspPoseToPreGrasp(X_WGrasp):
    X_GPregrasp = RigidTransform([0, -0.1, 0])
    X_WPregrasp = X_WGrasp @ X_GPregrasp
    return X_WPregrasp

def returnPose():
    X_WReturn = RigidTransform(RotationMatrix.MakeXRotation(-np.pi/2), [0.2, -0.3, 0.4])
    rotation = RigidTransform(RotationMatrix.MakeYRotation(np.pi/2))
    X_WReturn = X_WReturn @ rotation
    return X_WReturn

X_WGrasp = convertApplePoseToGrasp(X_WA)
X_WPregrasp = convertGraspPoseToPreGrasp(X_WGrasp)
X_WPluck = X_WPregrasp
X_WReturn = returnPose()

#AddMeshcatTriad(meshcat, "grasp", length=0.25, radius=0.01, X_PT=X_WGrasp)

iiwa_model = plant.GetModelInstanceByName("iiwa")
q0 = plant.GetPositions(plant_context, iiwa_model)

# Pregrasp
print("Pre")
traj_pregrasp = find_trajectory(q0, X_WPregrasp)
q_pregrasp = traj_pregrasp.value(traj_pregrasp.end_time())

# Grasp
print("Grasp")
traj_grasp = find_trajectory(q_pregrasp, X_WGrasp, 4)
q_grasp = traj_grasp.value(traj_grasp.end_time())
AddMeshcatTriad(meshcat, "grasp", length=0.25, radius=0.01, X_PT=X_WGrasp)

# Hold
print("Hold")
traj_hold = PiecewisePolynomial.FirstOrderHold([0., 0.5], np.array([q_grasp.squeeze(), q_grasp.squeeze()]).T)
q_hold = traj_hold.value(traj_hold.end_time())

# Pluck
print("Pluck")
traj_pluck = find_trajectory(q_hold, X_WPluck, 4)
q_pluck = traj_pluck.value(traj_pluck.end_time())

# Return
print("Return")
traj_return = find_trajectory(q_pluck, X_WReturn, q_guess_final = q0)
q_return = traj_return.value(traj_return.end_time())
AddMeshcatTriad(meshcat, "return", length=0.25, radius=0.01, X_PT=X_WReturn)

#Drop
print("Drop")
traj_drop = PiecewisePolynomial.FirstOrderHold([0., 1.], np.array([q_return.squeeze(), q_return.squeeze()]).T)

trajectories = [traj_pregrasp, traj_grasp, traj_hold, traj_pluck, traj_return, traj_drop]
grasp_states = [False, False, True, True, True, False]


def concatenate_trajectories(trajectories):
    if len(trajectories) == 0:
        return initial_trajectory

    adjusted_trajectories = [initial_trajectory]
    previous_end = initial_trajectory.end_time()
    intervals = [[0, previous_end]]

    for trajectory in trajectories:
        trajectory_end = trajectory.end_time()
        new_end = previous_end + trajectory_end
        print([previous_end, new_end])
        print(np.array([[0], [trajectory_end]]).T)
        intervals.append([previous_end, new_end])
        time_shift = PiecewisePolynomial.FirstOrderHold([previous_end, new_end], np.array([[0], [trajectory_end]]).T)
        adjusted_trajectory = PathParameterizedTrajectory(trajectory, time_shift)
        adjusted_trajectories.append(adjusted_trajectory)
        previous_end = new_end

    return CompositeTrajectory(adjusted_trajectories), intervals

def generate_gripper_trajectories(grasp_states, intervals):
    
    if len(grasp_states) == 0:
        return initial_g_traj

    composite_trajectories = [initial_g_traj]
    for interval, grasp_state in zip(intervals[1:], grasp_states):
        if grasp_state == False:
            grasp_val = 0.1
        else:
            grasp_val = -0.05
        print(interval)
        gripper_trajectory = PiecewisePolynomial.FirstOrderHold(interval, np.array([[grasp_val], [grasp_val]]).T)
        composite_trajectories.append(gripper_trajectory)
    return CompositeTrajectory(composite_trajectories)
        
    

combined_trajectory, intervals = concatenate_trajectories(trajectories)
combined_gripper_trajectory = generate_gripper_trajectories(grasp_states, intervals)

    

Pre
10
(7, 10)
Grasp
4
(7, 4)
Hold
Pluck
4
(7, 4)
Return
10
(7, 10)
Drop
[3.0, 4.28453675410155]
[[0.         1.28453675]]
[4.28453675410155, 5.28453675410155]
[[0. 1.]]
[5.28453675410155, 5.78453675410155]
[[0.  0.5]]
[5.78453675410155, 6.78453675410155]
[[0. 1.]]
[6.78453675410155, 8.132491501457555]
[[0.         1.34795475]]
[8.132491501457555, 9.132491501457555]
[[0. 1.]]
[3.0, 4.28453675410155]
[4.28453675410155, 5.28453675410155]
[5.28453675410155, 5.78453675410155]
[5.78453675410155, 6.78453675410155]
[6.78453675410155, 8.132491501457555]
[8.132491501457555, 9.132491501457555]


In [82]:
end_t = combined_trajectory.end_time()
q_traj_system.UpdateTrajectory(combined_trajectory)
g_traj_system.UpdateTrajectory(combined_gripper_trajectory)
#meshcat.StartRecording()
simulator.AdvanceTo(end_t)
meshcat.PublishRecording()

In [ ]:
new_len = new_traj.end_time()
t1 = initial_trajectory.end_time()
points = np.array([[0], [new_len]])
time_shift = PiecewisePolynomial.FirstOrderHold([t1, new_len+t1],points.T)
shifted_new = PathParameterizedTrajectory(new_traj, time_shift)

composite_trajectory = CompositeTrajectory([initial_trajectory, shifted_new])
t = composite_trajectory.end_time()
print(t)
q_traj_system.UpdateTrajectory(composite_trajectory)
meshcat.StartRecording()
simulator.AdvanceTo(t+1)
meshcat.PublishRecording()

result = composite_trajectory.value(t)
print(result)
print(new_traj.value(new_len))

In [ ]:
t_end = new_traj.end_time()
val_end = new_traj.value(t_end)
newt = PiecewisePolynomial(val_end)
q_traj_system.UpdateTrajectory(newt)
meshcat.StartRecording()
simulator.AdvanceTo(10)
meshcat.PublishRecording()

In [ ]:
iiwa_model = plant.GetModelInstanceByName("iiwa")

port = q_traj_system.GetOutputPort("y0")
print(port.Eval(q_traj_system_context))

q0 = plant.GetPositions(plant_context, iiwa_model)
print(q0)


print(composite_trajectory.value(t+10))


In [ ]:
def create_q_knots(q0, X_WA, visualize = False):
    """Convert end-effector pose list to joint position list using series of
    InverseKinematics problems. Note that q is 9-dimensional because the last 2 dimensions
    contain gripper joints, but these should not matter to the constraints.
    @param: pose_lst (python list): post_lst[i] contains keyframe X_WG at index i.
    @return: q_knots (python_list): q_knots[i] contains IK solution that will give f(q_knots[i]) \approx pose_lst[i].
    """
    q_knots = []
    plant, visualizer, diagram = CreateIiwaControllerPlant(visualize)
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    world_frame = plant.world_frame()
    gripper_frame = plant.GetFrameByName("body")
    q_nominal = np.array(
        [0.0, 0.6, 0.0, -1.75, 0.0, 1.0, 0.0]
    )  # nominal joint angles for joint-centering.

    def AddOrientationConstraint(ik, R_WG, bounds):
        """Add orientation constraint to the ik problem. Implements an inequality
        constraint where the axis-angle difference between f_R(q) and R_WG must be
        within bounds. Can be translated to:
        ik.prog().AddBoundingBoxConstraint(angle_diff(f_R(q), R_WG), -bounds, bounds)
        """
        ik.AddOrientationConstraint(
            frameAbar=world_frame,
            R_AbarA=R_WG,
            frameBbar=gripper_frame,
            R_BbarB=RotationMatrix(),
            theta_bound=bounds,
        )

    def AddPositionConstraint(ik, p_WG_lower, p_WG_upper):
        """Add position constraint to the ik problem. Implements an inequality
        constraint where f_p(q) must lie between p_WG_lower and p_WG_upper. Can be
        translated to
        ik.prog().AddBoundingBoxConstraint(f_p(q), p_WG_lower, p_WG_upper)
        """
        ik.AddPositionConstraint(
            frameA=world_frame,
            frameB=gripper_frame,
            p_BQ=np.zeros(3),
            p_AQ_lower=p_WG_lower,
            p_AQ_upper=p_WG_upper,
        )

    pose_lst = [X_WA]
    q_nominal = q0
    for i in range(len(pose_lst)):
        ik = inverse_kinematics.InverseKinematics(plant)
        q_variables = ik.q()  # Get variables for MathematicalProgram
        prog = ik.prog()  # Get MathematicalProgram

        #### Modify here ###############################

        # Initial Guess
        if i == 0:
            prog.SetInitialGuess(q_variables, q_nominal)
        else:
            prog.SetInitialGuess(q_variables, q_knots[-1])
   
        # L2 Squared Cost
        prog.AddQuadraticErrorCost(np.eye(len(q_variables)), q_nominal, q_variables)

        X_WG = pose_lst[i]

        # Position Constraint
        p_WG = X_WG.translation()
        d = 0.01
        p_WG_lower = p_WG - d
        p_WG_upper = p_WG + d
        AddPositionConstraint(ik, p_WG_lower, p_WG_upper)

        # Rotation Constraint
        R_WG = X_WG.rotation()
        AddOrientationConstraint(ik, R_WG, np.pi/180)

        ################################################

        result = Solve(prog)

        assert result.is_success()
        solution = result.GetSolution(q_variables)
        q_knots.append(solution)

    q_knots = np.array(q_knots)
    print(q_knots.shape)
    q_knots = np.vstack([q0, q_knots])
    trajectory = PiecewisePolynomial.FirstOrderHold([0, 2], q_knots[:, 0:7].T)
    if (visualize):  
        PublishPositionTrajectory(
            trajectory, context, plant, visualizer
        )
    
    return trajectory

#gripper_frame = self.plant.GetFrameByName("body")
gripper_frame = plant.GetFrameByName("iiwa_link_7")
world_frame = plant.world_frame()

iiwa_model = plant.GetModelInstanceByName("iiwa")
q0 = plant.GetPositions(plant_context, iiwa_model)
new_traj = create_q_knots(q0, X_WA, True)


In [ ]:
# Graveyard

iiwa_plant = plant.GetModelInstanceByName("iiwa")
q0 = plant.GetPositions(plant_context)
n = plant.num_model_instances()
for i in range(n):
    idx = ModelInstanceIndex(i)
    model_name = plant.GetModelInstanceName(idx)
    print(model_name)

#######


#differential_ik.GetInputPort("X_WE_desired").FixValue(differential_context, X_WA)
print(X_WA)

####

simulator.set_target_realtime_rate(1)
meshcat.AddButton("Stop Simulation", "Escape")
print("Press Escape to stop the simulation")
while meshcat.GetButtonClicks("Stop Simulation") < 1:
    print("Simulating")
    simulator.AdvanceTo(simulator.get_context().get_time() + 2)
meshcat.DeleteButton("Stop Simulation")

#####

meshcat.StartRecording()
simulator.AdvanceTo(4.0)
meshcat.PublishRecording()

###
plant_robot.GetJointByName("iiwa_joint_7").set_position_limits(
            [-np.inf], [np.inf]
)
###

# Old Hardware Station Approach
'''
scenario = load_scenario(data=scenario_data)
station = builder.AddSystem(MakeHardwareStation(scenario, meshcat))
controller_plant = station.GetSubsystemByName(
        "iiwa.controller"
).get_multibody_plant_for_control()
differential_ik = AddIiwaDifferentialIK(
    builder,
    controller_plant,
    frame=controller_plant.GetFrameByName("iiwa_link_7"), # should be "body" for wsg i think
)
# Nothing is connected to controller.get_input_port(0)
builder.Connect(
    differential_ik.get_output_port(),
    station.GetInputPort("iiwa.position"),
)
builder.Connect(
    station.GetOutputPort("iiwa.state_estimated"),
    differential_ik.GetInputPort("robot_state"),
)
'''

###
'''
# Teleop
meshcat.DeleteAddedControls()
teleop = builder.AddSystem(
    MeshcatPoseSliders(
        meshcat,
        lower_limit=[0, -np.pi, -np.pi, -0.6, -1, 0],
        upper_limit=[2 * np.pi, np.pi, np.pi, 0.8, 1, 1.1],       
    )
)
builder.Connect(
    teleop.get_output_port(), differential_ik.GetInputPort("X_WE_desired")
)
plant = station.GetSubsystemByName("plant")
ee_pose = builder.AddSystem(
    ExtractBodyPose(
        station.GetOutputPort("body_poses"),
        plant.GetBodyByName("iiwa_link_7").index(),
    )
)
builder.Connect(
    station.GetOutputPort("body_poses"), ee_pose.get_input_port()
)
builder.Connect(ee_pose.get_output_port(), teleop.get_input_port())

wsg_teleop = builder.AddSystem(WsgButton(meshcat))
builder.Connect(
    wsg_teleop.get_output_port(0), station.GetInputPort("wsg.position")
)
'''

#station.GetInputPort("wsg_position").FixValue(plant_context, [0.1, 0.1])

#differential_context = differential_ik.GetMyContextFromRoot(diagram_context)
#fixed_value = differential_ik.GetInputPort("X_WE_desired").FixValue(differential_context, RigidTransform([0, 0, 0]))

#plant.get_actuation_input_port().FixValue(plant_context, np.zeros(9))

####

gripper_frame = plant.GetFrameByName("iiwa_link_7")
world_frame = plant.world_frame()

def get_X_WG():
    X_WG = plant.CalcRelativeTransform(
        plant_context, frame_A=world_frame, frame_B=gripper_frame
    )
    return X_WG